# Codemao-Studio-Ranking-List

编程猫工作室评论数排行榜 Python 脚本，由黄色鸳尾编写，由 Hatmic 维护。

### 如何使用

请确保你已经正确安装 Python，如果没有安装，请到 [Python 官网](https://python.org) 中下载合适的版本并将 Python 添加到环境变量。

你需要安装 Python 的 requests 库，如果你没有安装，你需要在命令行中输入 `pip install requests`，等待其安装完成。

你可以在 [GitHub 仓库](https://github.com/Hatmic/Codemao-Studio-Ranking) 中获取源码，具体地，你可以点击 Code，点击 Download Zip。文件下载到本地并解压缩后，运行 main.py 文件。

结果爬取出来需要一定的时间，请耐心等待。

In [ ]:
import requests
import json
import time
import threading
c=time.time()
class Count_Replies:
    def __init__(self):
        self.data=self.get_data()

    def get_max_id(self):
        w=requests.get("https://api.codemao.cn/web/work-shops/search?limit=1&sort=-created_at")
        data=json.loads(w.text)
        return data["items"][0]["id"]

    def get_data(self):
        n=self.get_max_id()
        lst=[]
        num = min(200,n)
        link_range_list = [(int(i * (n) / num), -1+int((1 + i) * (n) / num)) for i in range(num)]
        # print(link_range_list)

        thread_list = []
        for i in range(1, num + 1):
            thread = myThread("Thread-" + str(i), link_range_list[i - 1])
            thread.start()
            thread_list.append(thread)
        # print(len(thread_list))
        for i in thread_list:
            i.join()
        flag=all([i.flag for i in thread_list])
        while not flag:
            flag = all([i.flag for i in thread_list])
        for i in thread_list:
            lst+=i.lst
        # print(lst)
        # print(len(lst))
        return lst
    def show_list(self):
        lst=[[i["all"],i["name"],i["id"]] for i in self.data]
        lst.sort()
        lst=lst[::-1]
        return lst
class myThread(threading.Thread):
    def __init__(self, name, link_range):
        threading.Thread.__init__(self)
        self.name = name
        self.link_range = link_range
        self.lst = []
        self.flag=False

    def run(self):
        # print("Starting" + self.name)
        self.crawler(self.name, self.link_range)
        # print("Exiting" + self.name)

    def crawler(self, link_num, link_range):
        self.lst=[]
        for i in range(link_range[0],link_range[1]+1):
            # if link_num=="Thread-1":
                # print(i/len(link_range))
            l={}
            try:
                w = requests.get(f"https://api.codemao.cn/web/shops/{i}")
                l["name"] = json.loads(w.text)["name"]
            except:
                continue
            w=requests.get(f"https://api.codemao.cn/web/discussions/{i}/comments?source=WORK_SHOP&limit=5&offset=0")
            data = json.loads(w.text)
            l["reply"]=data["total"]#评论
            l["all"]=data["totalReply"]+data["total"]#总数，包括评论回复
            l["id"]=i
            self.lst.append(l)
        self.flag=True

if __name__=="__main__":
    count=Count_Replies()

    print(count.show_list())#[数量,名称,工作室id]

    print(time.time()-c)
